<a href="https://colab.research.google.com/github/amrahmani/Marketing/blob/main/AIMarketing_Ch1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Read Kaggle Datasets in Google Colab**

**1. Get Your Kaggle API Token:**

Go to the Kaggle website (https://www.kaggle.com/) and log in to your account.
Navigate to your account settings by clicking on your profile picture in the top right corner and selecting "Account."
Click on the “Settings” button.
Scroll down to the "API" section. Click on the "Create New API Token" button.
This will download a file named kaggle.json to your computer. This file contains your Kaggle API credentials.

**2. Upload Your Kaggle API Token to Google Colab:**

Open your Google Colab notebook. In the left sidebar, click on the "Files" icon.
Click the "Upload" button.
Select the kaggle.json file that you downloaded and upload it to the Colab environment.

**3. Install the Kaggle API Client in Colab:**

In a code cell in your Colab notebook, run the following command to install the Kaggle API client:

In [1]:
!pip install -q kaggle

**4. Configure the Kaggle API Credentials**:

In a new code cell, run the following commands to create the .kaggle directory and move the kaggle.json file into it with the correct permissions:

In [2]:
!mkdir -p ~/kaggle
!mv kaggle.json ~/kaggle/   # or !mv /content/kaggle.json ~/kaggle/
!chmod 600 ~/kaggle/kaggle.json

mv: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/kaggle/kaggle.json': No such file or directory


In [37]:
!ls -al ~/.kaggle/kaggle.json

-rw------- 1 root root 73 Apr 27 06:55 /root/.kaggle/kaggle.json


**5. Download the Dataset from Kaggle:**

You need to know the name of the dataset as it appears on Kaggle. This usually follows the format username/dataset-name.
Go to the Kaggle dataset page you want to use. The dataset name is typically found below the dataset title (e.g., vijayuv/onlineretail).
In a new code cell in Colab, use the kaggle datasets download command followed by the dataset name and the -p flag to specify the directory where you want to download the files (e.g., ./data/).

In [28]:
!kaggle datasets download -d vijayuv/onlineretail -p ./data/

Dataset URL: https://www.kaggle.com/datasets/vijayuv/onlineretail
License(s): CC0-1.0
 69% 5.00M/7.20M [00:00<00:00, 9.65MB/s]
100% 7.20M/7.20M [00:00<00:00, 9.37MB/s]


In [36]:
!ls -al ./data/

total 51896
drwxr-xr-x 2 root root     4096 Apr 27 07:06 .
drwxr-xr-x 1 root root     4096 Apr 27 07:04 ..
-rw-r--r-- 1 root root 45580638 Sep 21  2019 OnlineRetail.csv
-rw-r--r-- 1 root root  7548702 Sep 21  2019 onlineretail.zip


**6. Unzip the Dataset (if necessary):**

Most Kaggle datasets are downloaded as zip files. You'll need to unzip them to access the individual data files (like CSV files).

**7. Read the Data into Pandas:**

Once the dataset is unzipped, you can use the pandas library to read the data files (e.g., CSV files) into a DataFrame:

In [31]:
!unzip ./data/onlineretail.zip -d ./data/

Archive:  ./data/onlineretail.zip
  inflating: ./data/OnlineRetail.csv  


In [38]:
import pandas as pd
# Specify the encoding explicitly, for example 'latin1' or 'ISO-8859-1'
df = pd.read_csv('./data/OnlineRetail.csv', encoding='latin1')
# Now you can work with the DataFrame 'df'
print(df.head())

  InvoiceNo StockCode                          Description  Quantity  \
0    536365    85123A   WHITE HANGING HEART T-LIGHT HOLDER         6   
1    536365     71053                  WHITE METAL LANTERN         6   
2    536365    84406B       CREAM CUPID HEARTS COAT HANGER         8   
3    536365    84029G  KNITTED UNION FLAG HOT WATER BOTTLE         6   
4    536365    84029E       RED WOOLLY HOTTIE WHITE HEART.         6   

      InvoiceDate  UnitPrice  CustomerID         Country  
0  12/1/2010 8:26       2.55     17850.0  United Kingdom  
1  12/1/2010 8:26       3.39     17850.0  United Kingdom  
2  12/1/2010 8:26       2.75     17850.0  United Kingdom  
3  12/1/2010 8:26       3.39     17850.0  United Kingdom  
4  12/1/2010 8:26       3.39     17850.0  United Kingdom  
